1 - Import libraries

In [110]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import plotly.express as px
import plotly as py


2 - Read dataset

In [111]:
df = pd.read_csv('netflix_titles.csv')

3 - Understand dataset

In [112]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [114]:
df['show_id'].nunique()

8807

Since the dataset has 8807 lines, the show_id has uniques values
Lets remove the duplicates

In [115]:
df = df.drop_duplicates()

We found some null cells, especially in the director colunm. Lets fill all N/A with the string "unkown"

In [116]:
df.fillna("Unknow", inplace=True)

In [117]:
df.isnull().sum()

,0
show_id,0
type,0
title,0
director,0
cast,0
country,0
date_added,0
release_year,0
rating,0
duration,0


Done! Lets treat a little bit more about our dataset! There is a colunm with date_added but its type is an object, we should change the type to a datetime type:

In [118]:
df['date_added'] = pd.to_datetime(df['date_added'], errors = 'coerce')

Great, now we have a clean dataset! Now we can use the visualizations libraries to take insights!
To start, which questions we want to answer?


# How is the netflix portfolio?

In [119]:
df_types = df.groupby('type')['show_id'].count().reset_index()

In [120]:
fig = px.bar(df_types, x='type', y='show_id', text = 'show_id',color_discrete_sequence=['lightblue'])
fig.update_layout(
    title = 'Types of Netflix Portfolio',
    title_x=0.5,
    font = dict(family='Sans-Serif'),
    plot_bgcolor = 'white',
    yaxis_title = 'Number of titles',
    xaxis_title = 'Type',
)

fig.update_traces(width=0.4)
fig.update_xaxes(showline=True, linewidth=1, linecolor='black')
fig.update_yaxes(showticklabels=False)
fig.show()

# Which year netflix added more titles on the catalogue?

In [121]:
df_dateadded = df.groupby('date_added')['show_id'].count().reset_index()

In [122]:
fig1 = px.histogram(df_dateadded, x='date_added',color_discrete_sequence=['rosybrown'])
fig1.update_layout(
    title = 'Number of titles added per year',
    title_x=0.5,
    font = dict(family='Sans-Serif'),
    plot_bgcolor = 'white',
    yaxis_title = 'Number of titles added',
    xaxis_title = 'Year',
    bargap=0.2
)

fig1.update_xaxes(showline=True, linewidth=1, linecolor='black')
fig1.update_yaxes(showline=True, linewidth=1, linecolor='black')
fig1.update_xaxes(showgrid=False)
fig1.update_yaxes(showgrid=True, gridcolor='lightgray', griddash='dot')

fig1.show()

We can see that from the end of 2016 ahead, the numbers os titles added increased, with the maximum between April 2019 to Jun 2019 with 87 titles added.

# Are the titles new or old?

In [123]:
df_releasey = df.groupby('release_year')['show_id'].count().reset_index()

In [124]:
fig2 = px.bar(df_releasey, x='release_year', y='show_id',color_discrete_sequence=['peru'])
fig2.update_layout(
    title = 'Number of titles added per release tear',
    title_x=0.5,
    font = dict(family='Sans-Serif'),
    plot_bgcolor = 'white',
    yaxis_title = 'Number of titles',
    xaxis_title = 'Release Year'
)

fig2.update_xaxes(showline=True, linewidth=1, linecolor='black')
fig2.update_yaxes(showline=True, linewidth=1, linecolor='black')
fig2.update_xaxes(showgrid=False)
fig2.update_yaxes(showgrid=True, gridcolor='lightgray', griddash='dot')

fig2.show()

The netflix catalogue has more newest titles, especially the ones released in 2018

# Are most of titles from any country?

Lets look into de country colunm to indentify if we need to treat the data:



There are some cells that contain more than one country separated with ",", lets separate the country into new lines

In [125]:
df['country'] = df['country'].str.split(',')

In [126]:
df_expanded = df.explode('country', ignore_index=True)

Lets see if we have to do something else?

In [127]:
df_expanded['country'].unique()

array(['United States', 'South Africa', 'Unknow', 'India', ' Ghana',
       ' Burkina Faso', ' United Kingdom', ' Germany', ' Ethiopia',
       'United Kingdom', 'Germany', ' Czech Republic', 'Mexico', 'Turkey',
       'Australia', ' India', ' France', 'Finland', 'China', ' Canada',
       ' United States', ' Japan', 'Nigeria', 'Japan', 'Spain', 'France',
       'Belgium', 'South Korea', ' Singapore', ' Australia', ' Mexico',
       ' Italy', ' Romania', 'Argentina', ' Venezuela', ' Hong Kong',
       'Russia', 'Canada', 'Hong Kong', ' China', 'Italy', '',
       ' South Korea', 'Ireland', ' Nepal', 'New Zealand', ' Brazil',
       ' Greece', 'Jordan', 'Colombia', 'Switzerland', 'Israel', 'Brazil',
       ' Spain', 'Taiwan', ' Nigeria', 'Bulgaria', ' Algeria', 'Poland',
       ' Israel', 'Saudi Arabia', 'Thailand', 'Indonesia', 'Egypt',
       ' Denmark', ' Switzerland', 'Kuwait', ' Netherlands', ' Belgium',
       'Malaysia', ' New Zealand', 'Vietnam', ' Hungary', 'Sweden',
       'Le

It seems that we still have some adjustment to do. Some of the countries have a space before the string:

In [128]:
df_expanded['country'] = df_expanded['country'].str.strip().str.title()

In [129]:
mapa_paises = {
    'West Germany': 'Germany',
    'East Germany': 'Germany',
    'Soviet Union': 'Russia',
    'Puerto Rico': 'United States',
    'Vatican City': 'Holy See',
    'unknow': 'Unknown'
}
df_expanded['country'] = df_expanded['country'].replace(mapa_paises)

In [130]:
df_expanded['country'].unique()

array(['United States', 'South Africa', 'Unknow', 'India', 'Ghana',
       'Burkina Faso', 'United Kingdom', 'Germany', 'Ethiopia',
       'Czech Republic', 'Mexico', 'Turkey', 'Australia', 'France',
       'Finland', 'China', 'Canada', 'Japan', 'Nigeria', 'Spain',
       'Belgium', 'South Korea', 'Singapore', 'Italy', 'Romania',
       'Argentina', 'Venezuela', 'Hong Kong', 'Russia', '', 'Ireland',
       'Nepal', 'New Zealand', 'Brazil', 'Greece', 'Jordan', 'Colombia',
       'Switzerland', 'Israel', 'Taiwan', 'Bulgaria', 'Algeria', 'Poland',
       'Saudi Arabia', 'Thailand', 'Indonesia', 'Egypt', 'Denmark',
       'Kuwait', 'Netherlands', 'Malaysia', 'Vietnam', 'Hungary',
       'Sweden', 'Lebanon', 'Syria', 'Philippines', 'Iceland',
       'United Arab Emirates', 'Norway', 'Qatar', 'Mauritius', 'Austria',
       'Cameroon', 'Palestine', 'Uruguay', 'Kenya', 'Chile', 'Luxembourg',
       'Cambodia', 'Bangladesh', 'Portugal', 'Cayman Islands', 'Senegal',
       'Serbia', 'Malta', 'Na

Great! now we have a smaller list that doesnt need more treatment.

In [131]:
!pip install pycountry

In [132]:
import pycountry

def obter_iso3(nome_pais):
    try:
        country = pycountry.countries.get(name=nome_pais)
        return country.alpha_3 if country else None
    except KeyError:
        return None

df_expanded['country_iso3'] = df_expanded['country'].apply(obter_iso3)

print(df_expanded['country_iso3'].unique())


['USA' 'ZAF' None 'IND' 'GHA' 'BFA' 'GBR' 'DEU' 'ETH' 'MEX' 'AUS' 'FRA'
 'FIN' 'CHN' 'CAN' 'JPN' 'NGA' 'ESP' 'BEL' 'SGP' 'ITA' 'ROU' 'ARG' 'HKG'
 'IRL' 'NPL' 'NZL' 'BRA' 'GRC' 'JOR' 'COL' 'CHE' 'ISR' 'BGR' 'DZA' 'POL'
 'SAU' 'THA' 'IDN' 'EGY' 'DNK' 'KWT' 'NLD' 'MYS' 'HUN' 'SWE' 'LBN' 'PHL'
 'ISL' 'ARE' 'NOR' 'QAT' 'MUS' 'AUT' 'CMR' 'URY' 'KEN' 'CHL' 'LUX' 'KHM'
 'BGD' 'PRT' 'CYM' 'SEN' 'SRB' 'MLT' 'NAM' 'AGO' 'PER' 'MOZ' 'BLR' 'ZWE'
 'PAK' 'CYP' 'GTM' 'IRQ' 'MWI' 'PRY' 'HRV' 'ALB' 'GEO' 'MAR' 'SVK' 'UKR'
 'BMU' 'ECU' 'ARM' 'MNG' 'BHS' 'LKA' 'LVA' 'LIE' 'CUB' 'NIC' 'SVN' 'DOM'
 'WSM' 'AZE' 'BWA' 'JAM' 'KAZ' 'LTU' 'AFG' 'SOM' 'SDN' 'PAN' 'UGA' 'MNE']


Now lets visualize our data:

In [133]:
df_country = df_expanded.groupby('country_iso3')['show_id'].count().reset_index().sort_values(by=['show_id'],ascending=False)

In [134]:
df_country10 = df_country.head(10)

In [135]:
fig3 = px.bar(df_country10, x='country_iso3', y='show_id', text = 'show_id', color_discrete_sequence=['darkblue'])
fig3.update_layout(
    title = 'Number of titles per country',
    title_x=0.5,
    font = dict(family='Sans-Serif'),
    plot_bgcolor = 'white',
    xaxis_title = 'Country of origin',
    yaxis_title = 'Number of titles'
)

fig3.update_yaxes(showticklabels=False)
fig3.update_traces(textfont_size = 10, textposition='outside')
fig3.show()

Most of the titles are from USA, then India and United Kingdom

Lets see all the countries from which  Netflix has movies or tv shows. To do it, I want to use the

In [136]:
fig4 = px.choropleth(df_country, locations="country_iso3",
                    color="show_id",
                    hover_name="country_iso3",
                    )

fig4.show()

As we can see, most of the titles are indeed from the USA, but we also have movies almost around all the countries, but there are some with no titles at all, like Russia and some countries in Africa

# What about the rating?

The definitions are:

*   TV-MA: Mature Audience
*   TV-14:	Suitable for 14+
*   TV-PG:	Parental Guidance Suggested
*   R:	Restricted
*   PG-13:	Parents Strongly Cautioned
*   TV-Y7:	Suitable for children age 7 and older
*   TV-Y:	All Children
*   PG:	Parental Guidance Suggested
*   TV-G:	General Audience
*   NR:	Not Rated
*   G:	General Audience
*   TV-Y7-FV:	Suitable for children age 7 and older with Fantasy Violence
*   NC-17:	No One 17 and Under Admitted
*   UR:	Unrated


In [137]:
df['rating'].unique()

array(['PG-13', 'TV-MA', 'PG', 'TV-14', 'TV-PG', 'TV-Y', 'TV-Y7', 'R',
       'TV-G', 'G', 'NC-17', '74 min', '84 min', '66 min', 'NR', 'Unknow',
       'TV-Y7-FV', 'UR'], dtype=object)

There are some data that should be in other colunm, lets identify if we have the same mistake in other column

In [138]:
df['release_year'].unique()

array([2020, 2021, 1993, 2018, 1996, 1998, 1997, 2010, 2013, 2017, 1975,
       1978, 1983, 1987, 2012, 2001, 2014, 2002, 2003, 2004, 2011, 2008,
       2009, 2007, 2005, 2006, 1994, 2015, 2019, 2016, 1982, 1989, 1990,
       1991, 1999, 1986, 1992, 1984, 1980, 1961, 2000, 1995, 1985, 1976,
       1959, 1988, 1981, 1972, 1964, 1945, 1954, 1979, 1958, 1956, 1963,
       1970, 1973, 1925, 1974, 1960, 1966, 1971, 1962, 1969, 1977, 1967,
       1968, 1965, 1946, 1942, 1955, 1944, 1947, 1943])

In [139]:
df['duration'].unique()

array(['90 min', '2 Seasons', '1 Season', '91 min', '125 min',
       '9 Seasons', '104 min', '127 min', '4 Seasons', '67 min', '94 min',
       '5 Seasons', '161 min', '61 min', '166 min', '147 min', '103 min',
       '97 min', '106 min', '111 min', '3 Seasons', '110 min', '105 min',
       '96 min', '124 min', '116 min', '98 min', '23 min', '115 min',
       '122 min', '99 min', '88 min', '100 min', '6 Seasons', '102 min',
       '93 min', '95 min', '85 min', '83 min', '113 min', '13 min',
       '182 min', '48 min', '145 min', '87 min', '92 min', '80 min',
       '117 min', '128 min', '119 min', '143 min', '114 min', '118 min',
       '108 min', '63 min', '121 min', '142 min', '154 min', '120 min',
       '82 min', '109 min', '101 min', '86 min', '229 min', '76 min',
       '89 min', '156 min', '112 min', '107 min', '129 min', '135 min',
       '136 min', '165 min', '150 min', '133 min', '70 min', '84 min',
       '140 min', '78 min', '7 Seasons', '64 min', '59 min', '139 min',
    

It doesnt seem like. Lets see which line has this problem

In [140]:
df_problem = df.loc[df['rating'].isin(["74 min","84 min","66 min"])]

It seems that these lines dont have rating, and the duration is in the wrong colunm, lets adjust:

In [141]:
df.loc[5541, 'duration'] = df.loc[5541, 'rating']
df.loc[5794, 'duration'] = df.loc[5794, 'rating']
df.loc[5813, 'duration'] = df.loc[5813, 'rating']

In [142]:
df.loc[5541, 'rating']= "unknow"
df.loc[5794, 'rating']= "unknow"
df.loc[5813, 'rating']= "unknow"

Now lets visualize:

In [143]:
df_rating = df.groupby('rating')['show_id'].count().reset_index().sort_values(by='show_id',ascending=False)

In [144]:
fig5 = px.bar(
    df_rating,
    x ='rating',
    y ='show_id',
    color_discrete_sequence=['lightblue'],
    text='show_id'
)
fig5.update_layout(
    title='Number of titles per rating',
    title_x = 0.5,
    xaxis_title = 'Rating',
    yaxis_title = 'Number of titles',
    plot_bgcolor= 'white'
)

fig5.update_xaxes(showline=True, linewidth=1, linecolor='black')
fig5.update_yaxes(showticklabels=False)
fig5.update_traces(textfont_size = 10)

fig5.show()

As we can see, most of the titles are for mature audience (TV-MA), followed by the titles suitables for 14+.


# And what about the directors, do Netflix have a favorite?



In [145]:
df_directors = df.groupby('director')['show_id'].count().reset_index().sort_values(by='show_id',ascending=False)

In [146]:
df_directors10 = df_directors.loc[df_directors['director']!='Unknow']

In [147]:
df_directors10 = df_directors10.head(10)

In [148]:
fig6 = px.bar(df_directors10, x='director', y='show_id', text = 'show_id', color_discrete_sequence=['orange'])
fig6.update_layout(
    title = 'Top 10 directors with more titles',
    title_x=0.5,
    font = dict(family='Sans-Serif'),
    plot_bgcolor = 'white',
    xaxis_title = 'Director',
    yaxis_title = 'Number of titles'
)

fig6.update_yaxes(showticklabels=False)
fig6.update_traces(textfont_size = 12, textposition='inside')
fig6.show()

Interesting, the director with more movies on Netflix is Rajiv Chilaka, an indian director!
Great! Now lets include more databases to complete our analysis! In this data, we can find the IMDB score for netflix movies and tv shows

In [149]:
df_imdb = pd.read_csv('IMBD.csv')

In [150]:
df_imdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9957 entries, 0 to 9956
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   title        9957 non-null   object 
 1   year         9430 non-null   object 
 2   certificate  6504 non-null   object 
 3   duration     7921 non-null   object 
 4   genre        9884 non-null   object 
 5   rating       8784 non-null   float64
 6   description  9957 non-null   object 
 7   stars        9957 non-null   object 
 8   votes        8784 non-null   object 
dtypes: float64(1), object(8)
memory usage: 700.2+ KB


In [151]:
df_imdb = df_imdb.drop_duplicates()

In [152]:
df_imdb['title']

,title
0,Cobra Kai
1,The Crown
2,Better Call Saul
3,Devil in Ohio
4,Cyberpunk: Edgerunners
...,...
9952,The Imperfects
9953,The Walking Dead
9954,The Crown
9955,Supernatural


In [153]:
df_imdb.isnull().sum()

,0
title,0
year,527
certificate,3453
duration,2036
genre,73
rating,1173
description,0
stars,0
votes,1173


In [154]:
df_imdb.fillna('Unknow', inplace=True)

<ipython-input-154-aed860e5c9a2>:1: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Unknow' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.



In [155]:
df_imdb.drop_duplicates(inplace=True)

In [156]:
df_imdb['rating'] = pd.to_numeric(df_imdb['rating'], errors='coerce')

In [157]:
df_imdb = df_imdb.groupby('title')['rating'].mean().reset_index()

In [158]:
df = pd.merge(df, df_imdb[['title', 'rating']], on='title', how='left')

In [159]:
df.fillna('Unknow', inplace=True)

<ipython-input-159-3e8803bce560>:1: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Unknow' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.

<ipython-input-159-3e8803bce560>:1: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Unknow' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.



In [160]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      8807 non-null   object
 4   cast          8807 non-null   object
 5   country       8807 non-null   object
 6   date_added    8807 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating_x      8807 non-null   object
 9   duration      8807 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
 12  rating_y      8807 non-null   object
dtypes: int64(1), object(12)
memory usage: 894.6+ KB


# Now lets visualize

In [161]:
df_score = df.groupby('rating_y')['title'].count().reset_index()

In [162]:
df_score

,rating_y,title
0,2.0,1
1,2.3,1
2,2.4,1
3,2.5,1
4,2.6,2
...,...,...
228,9.111111,1
229,9.2,1
230,9.3,1
231,9.5,1


Since I got the IMBD dataset from kraggle and most of the titles arent there, lets ignore these titles to continue the analysis, but in a real analysis it wouldnt be right to use the conclusions from this

In [180]:
df_score = df_score.loc[df_score['rating_y']!='Unknow']

In [188]:
fig7 = px.bar(df_score, x='rating_y', y='title',color_discrete_sequence=['lightpink'])
fig7.update_layout(
    title = 'Number of titles per IMDB score',
    title_x=0.5,
    font = dict(family='Sans-Serif'),
    plot_bgcolor = 'white',
    yaxis_title = 'Number of titles',
    xaxis_title = 'IMDB Score',
    bargap = 0.1
)

fig7.update_xaxes(showline=True, linewidth=1, linecolor='black')
fig7.update_yaxes(showline=True, linewidth=1, linecolor='black')
fig7.update_xaxes(showgrid=False)
fig7.update_yaxes(showgrid=True, gridcolor='lightgray', griddash='dot')
fig7.update_traces(width=0.1)
fig7.show()

In [177]:
df_score['rating_y'].mean().round(1)

6.8

It seems that most of the titles has a rating between 7 and 8. But what is the mean rating of all movies from IMDB dataset?

In [167]:
df_imdb['rating'].mean().round(1)

6.5

It seems that the movies from Netflix portfolio are 0.3 better rated than all the movies rated in IMDB dataset.

We can do so many analysis with python, but now let's use this dataset to build a dashboard with Looker!

In [168]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      8807 non-null   object
 4   cast          8807 non-null   object
 5   country       8807 non-null   object
 6   date_added    8807 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating_x      8807 non-null   object
 9   duration      8807 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
 12  rating_y      8807 non-null   object
dtypes: int64(1), object(12)
memory usage: 894.6+ KB


In [169]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [170]:
df.to_excel(r"/content/drive/MyDrive/Projects/netflix_dataset2.xlsx", index=False)